In [37]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
import re

In [38]:
data = pd.read_json('../../events_meetup.json')


In [39]:
data.loc[:,'location'] = data.loc[:,'location'].str[0]

In [40]:
data.loc[:,'date'] = data.loc[:,'date'].str[0]

In [41]:
data.loc[:,'event_name'] = data.loc[:,'event_name'].str[0]

In [42]:
data

,event_name,date,location
0,Youth Summit,19 & 20 Sep | 3:00 PM,American University in Cairo
1,Myriam Fares at KIKI’s White,Sep 19 | 09:00 PM,"KIKI’s Beach, Hacienda White"
2,Crowd Gigs By Mohamed Helmy - Saturday Early Show,Sep 20 | 04:30 PM,Theatro Arkan
3,Layali Marassi: Asala X Ahmed Saad,Sep 20 | 11:00 PM,"Address Beach Resort, Marassi"
4,Comedy Therapy,Sep 23 | 09:00 PM,Cairo Jazz Club 610
5,Cityscape Egypt 2025 - VIP Experience,24th to 27th of September 2025,Egypt International Exhibition Center
6,20 years of Junior,Sep 25 | 09:00 PM,Cairo Jazz Club 610
7,Marwa Nagy - 50th anniversary of Umm Kalthoum,Sep 26 | 08:00 PM,Theatro Arkan
8,Cityscape Egypt 2025 - VIP Experience (Day 4),Sep 27 | 12:00 PM,Egypt International Exhibition Center
9,SheArts,From 2nd to 5th Oct,Concert Hall (New Capital) - AUC Tahrir Square...


In [43]:
data[['date','Time']] =  data.date.str.split("|", n=1, expand=True)
data.rename(columns={'location':'venue_name'},inplace=True)


In [44]:
def get_coordinates(row):
    geolocator = Nominatim(user_agent="my_geocoder")
    address = f"{row['venue_address']}, Egypt"
    location = geolocator.geocode(address)
    if location:
        return pd.Series([location.latitude, location.longitude])
    else:
        return None, None
def get_city(row):
    """uses coordinates to get city"""
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.reverse(row['venue_address'], language='en',zoom=10)  
    return location.raw['address'].get('city')

In [45]:
data.venue_name = data.venue_name.str.normalize('NFKD')
data

,event_name,date,venue_name,Time
0,Youth Summit,19 & 20 Sep,American University in Cairo,3:00 PM
1,Myriam Fares at KIKI’s White,Sep 19,"KIKI’s Beach, Hacienda White",09:00 PM
2,Crowd Gigs By Mohamed Helmy - Saturday Early Show,Sep 20,Theatro Arkan,04:30 PM
3,Layali Marassi: Asala X Ahmed Saad,Sep 20,"Address Beach Resort, Marassi",11:00 PM
4,Comedy Therapy,Sep 23,Cairo Jazz Club 610,09:00 PM
5,Cityscape Egypt 2025 - VIP Experience,24th to 27th of September 2025,Egypt International Exhibition Center,None
6,20 years of Junior,Sep 25,Cairo Jazz Club 610,09:00 PM
7,Marwa Nagy - 50th anniversary of Umm Kalthoum,Sep 26,Theatro Arkan,08:00 PM
8,Cityscape Egypt 2025 - VIP Experience (Day 4),Sep 27,Egypt International Exhibition Center,12:00 PM
9,SheArts,From 2nd to 5th Oct,Concert Hall (New Capital) - AUC Tahrir Square...,None


In [46]:
ref = pd.read_csv(r"E:\youssef ashmawy\programming projects\Python\Nokia Task\output\referance.csv")

In [47]:
ref.city = ref.city.str.lower()

In [48]:
ref

,event_name,venue_name,venue_address,date_part,time_part,city
0,Medfest Eg 7th Edition,AUC Tahrir Square,"30.04304943486499, 31.236530904482507",2025-09-18,10:30,cairo
1,Fouad and Mounib,Theatro Arkan,"30.020402636742446, 31.003588270211274",2025-09-18,20:00,giza
2,Layali Marassi: Ragheb Allama x Amina,"Address Beach Resort, Marassi","30.966378514279903, 28.764663160486883",2025-09-18,23:00,marsa matrouh
3,Youth Summit,American University in Cairo,"30.019224469626735, 31.502954970733693",2025-09-19,15:00,cairo
4,Crowd Gigs By Mohamed Helmy - Early Show,Theatro Arkan,"30.020402636742446, 31.003588270211274",2025-09-19,16:30,giza
5,Crowd Gigs By Mohamed Helmy - First Show,Theatro Arkan,"30.020402636742446, 31.003588270211274",2025-09-19,20:00,giza
6,Tá brón orm (Sorrow Is Upon Me),Rawabet Art Space,"30.049483336333587, 31.238033769313294",2025-09-19,20:00,cairo
7,Myriam Fares at KIKI’s White,"KIKI’s Beach, Hacienda White","31.002772740274153, 28.66518344233024",2025-09-19,21:00,marsa matrouh
8,Ain Gamal x7th Medfest EG,AUC Tahrir Square,"30.04304943486499, 31.236530904482507",2025-09-19,21:30,cairo
9,Layali Marassi: Tamer Ashour,"Address Beach Resort, Marassi","30.966378514279903, 28.764663160486883",2025-09-19,23:00,marsa matrouh


In [49]:
ref.set_index('venue_name')[['venue_address']].to_dict()

{'venue_address': {'AUC Tahrir Square': '30.04304943486499, 31.236530904482507',
  'Theatro Arkan': '30.020402636742446, 31.003588270211274',
  'Address Beach Resort, Marassi': '30.966378514279903, 28.764663160486883',
  'American University in Cairo': '30.019224469626735, 31.502954970733693',
  'Rawabet Art Space': '30.049483336333587, 31.238033769313294',
  'KIKI’s Beach, Hacienda White': '31.002772740274153, 28.66518344233024',
  'Boom Room, Open Air Mall, Madinaty': '30.108738710312494, 31.6265184785103',
  'St. Regis New Administrative Capital': '30.006395089931058, 31.723177913495356',
  'Cairo Jazz Club 610': '30.019941913960334, 31.00298474601855',
  'Egypt International Exhibition Center ': '30.0144858, 31.3842229',
  'The Greek Campus': '30.044839501411428, 31.238882713493425',
  'El Arena': '29.959543532076435, 31.450887869711284',
  'Galambo': '31.0034673117396, 28.6590109102205',
  'Falaki Mainstage Theater': '30.042409719866747, 31.23852597229928',
  'New Capital': '30.01

In [50]:
venue_mapping = ref.set_index('venue_name')['venue_address'].to_dict()
city_mapping =  ref.set_index('venue_name')['city'].to_dict()

In [51]:
city_mapping

{'AUC Tahrir Square': 'cairo',
 'Theatro Arkan': 'giza',
 'Address Beach Resort, Marassi': 'marsa matrouh',
 'American University in Cairo': 'cairo',
 'Rawabet Art Space': 'cairo',
 'KIKI’s Beach, Hacienda White': 'marsa matrouh',
 'Boom Room, Open Air Mall, Madinaty': 'cairo',
 'St. Regis New Administrative Capital': 'cairo',
 'Cairo Jazz Club 610': 'giza',
 'Egypt International Exhibition Center ': 'cairo',
 'The Greek Campus': 'cairo',
 'El Arena': 'cairo',
 'Galambo': 'marsa matrouh',
 'Falaki Mainstage Theater': 'cairo',
 'New Capital': 'cairo',
 'Concert Hall (New Capital) - AUC Tahrir Square - Falaki Mainstage Theater': 'cairo',
 'Movenpick Aswan': 'aswan',
 'Must Theatre': 'giza',
 'Port Said Culture Center ': 'port said',
 'Ewart Memorial Hall, AUC Tahrir Square': 'cairo',
 'El Rihany Theater': 'cairo',
 'Abdeen Palace': 'cairo',
 'The Theater': 'giza',
 'Al Anfoushy Theater': 'alexandria',
 'O West Orascom Development': 'giza',
 'Zed Park, Elsheikh Zayed': 'giza',
 'Concert H

In [52]:
venue_mapping

{'AUC Tahrir Square': '30.04304943486499, 31.236530904482507',
 'Theatro Arkan': '30.020402636742446, 31.003588270211274',
 'Address Beach Resort, Marassi': '30.966378514279903, 28.764663160486883',
 'American University in Cairo': '30.019224469626735, 31.502954970733693',
 'Rawabet Art Space': '30.049483336333587, 31.238033769313294',
 'KIKI’s Beach, Hacienda White': '31.002772740274153, 28.66518344233024',
 'Boom Room, Open Air Mall, Madinaty': '30.108738710312494, 31.6265184785103',
 'St. Regis New Administrative Capital': '30.006395089931058, 31.723177913495356',
 'Cairo Jazz Club 610': '30.019941913960334, 31.00298474601855',
 'Egypt International Exhibition Center ': '30.0144858, 31.3842229',
 'The Greek Campus': '30.044839501411428, 31.238882713493425',
 'El Arena': '29.959543532076435, 31.450887869711284',
 'Galambo': '31.0034673117396, 28.6590109102205',
 'Falaki Mainstage Theater': '30.042409719866747, 31.23852597229928',
 'New Capital': '30.013313382896914, 31.72227443852814

In [53]:
data['venue_coordinates'] = data['venue_name'].map(venue_mapping)
data['city'] = data['venue_name'].map(city_mapping)

In [54]:
data

,event_name,date,venue_name,Time,venue_coordinates,city
0,Youth Summit,19 & 20 Sep,American University in Cairo,3:00 PM,"30.019224469626735, 31.502954970733693",cairo
1,Myriam Fares at KIKI’s White,Sep 19,"KIKI’s Beach, Hacienda White",09:00 PM,"31.002772740274153, 28.66518344233024",marsa matrouh
2,Crowd Gigs By Mohamed Helmy - Saturday Early Show,Sep 20,Theatro Arkan,04:30 PM,"30.020402636742446, 31.003588270211274",giza
3,Layali Marassi: Asala X Ahmed Saad,Sep 20,"Address Beach Resort, Marassi",11:00 PM,"30.966378514279903, 28.764663160486883",marsa matrouh
4,Comedy Therapy,Sep 23,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza
5,Cityscape Egypt 2025 - VIP Experience,24th to 27th of September 2025,Egypt International Exhibition Center,None,"30.0144858, 31.3842229",cairo
6,20 years of Junior,Sep 25,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza
7,Marwa Nagy - 50th anniversary of Umm Kalthoum,Sep 26,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza
8,Cityscape Egypt 2025 - VIP Experience (Day 4),Sep 27,Egypt International Exhibition Center,12:00 PM,"30.0144858, 31.3842229",cairo
9,SheArts,From 2nd to 5th Oct,Concert Hall (New Capital) - AUC Tahrir Square...,None,"30.042333968763398, 31.238579755550326",cairo


In [55]:
# data.to_csv('TicketsMarche/ticketMarche.csv',index=False)

In [84]:
# Add this cell to your notebook to simplify date handling
def simplify_dates(df, date_column='date'):
    """
    Simplify date ranges by extracting just the first/start date.
    Returns DataFrame with standardized dates in YYYY-MM-DD format.
    """
    def extract_first_date(date_str):
        """Extract just the first/start date from any pattern"""
        if not isinstance(date_str, str) or not date_str.strip():
            return date_str
        

        
        # 29 sep or sep 29
        # from x to y return x
        day_match = re.search(r'(\d+)(?:st|nd|rd|th)*', date_str)
        month_match = re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*', 
                              date_str, re.IGNORECASE)
        
        if day_match and month_match:
            day = int(day_match.group(1))
            month_name = month_match.group(0)
            
            # Layali Marrasi, known date
            
            if month_name == 'Marassi':
                return datetime(2025, 9, 20).strftime('%Y-%m-%d')
            month_num = datetime.strptime(month_name[:3], '%b').month
            
            year_match = re.search(r'\b(20\d{2})\b', date_str)
            year = int(year_match.group(1)) if year_match else datetime.now().year
            
            try:
                return datetime(year, month_num, day).strftime('%Y-%m-%d')
            except ValueError:
                return date_str
        
        return date_str
    
    # Create a copy of the dataframe
    result_df = df.copy()
    
    # Apply the extraction function
    result_df[date_column] = result_df[date_column].apply(extract_first_date)
    
    # Add column to indicate original text
    result_df['original_date_text'] = df[date_column]
    
    return result_df

# Apply the simplified date extraction
simplified_data = simplify_dates(data)
print(f"Processed {len(simplified_data)} rows")

# Show the simplified dataframe
simplified_data

Processed 50 rows


,event_name,date,venue_name,Time,venue_coordinates,city,original_date_text
0,Youth Summit,2025-09-19,American University in Cairo,3:00 PM,"30.019224469626735, 31.502954970733693",cairo,19 & 20 Sep
1,Myriam Fares at KIKI’s White,2025-09-19,"KIKI’s Beach, Hacienda White",09:00 PM,"31.002772740274153, 28.66518344233024",marsa matrouh,Sep 19
2,Crowd Gigs By Mohamed Helmy - Saturday Early Show,2025-09-20,Theatro Arkan,04:30 PM,"30.020402636742446, 31.003588270211274",giza,Sep 20
3,Layali Marassi: Asala X Ahmed Saad,2025-09-20,"Address Beach Resort, Marassi",11:00 PM,"30.966378514279903, 28.764663160486883",marsa matrouh,Sep 20
4,Comedy Therapy,2025-09-23,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza,Sep 23
5,Cityscape Egypt 2025 - VIP Experience,2025-09-24,Egypt International Exhibition Center,None,"30.0144858, 31.3842229",cairo,24th to 27th of September 2025
6,20 years of Junior,2025-09-25,Cairo Jazz Club 610,09:00 PM,"30.019941913960334, 31.00298474601855",giza,Sep 25
7,Marwa Nagy - 50th anniversary of Umm Kalthoum,2025-09-26,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza,Sep 26
8,Cityscape Egypt 2025 - VIP Experience (Day 4),2025-09-27,Egypt International Exhibition Center,12:00 PM,"30.0144858, 31.3842229",cairo,Sep 27
9,SheArts,2025-10-02,Concert Hall (New Capital) - AUC Tahrir Square...,None,"30.042333968763398, 31.238579755550326",cairo,From 2nd to 5th Oct


In [89]:
year = 2025
month_num = 10
day = 10
simplified_data[simplified_data['date']>datetime(year, month_num, day).strftime('%Y-%m-%d')]

,event_name,date,venue_name,Time,venue_coordinates,city,original_date_text
15,Saleh El-Nawawy X Boom Room,2025-10-13,"Boom Room, Open Air Mall, Madinaty",09:00 PM,"30.108738710312494, 31.6265184785103",cairo,Oct 13
32,Oliver by Fabrica X Theatro Arkan,2025-10-16,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza,Oct 16 to Oct 18
33,Ibrahim Maalouf,2025-12-20,Concert Hall - New Capital,07:00 PM,"30.015641304795817, 31.72162606506574",cairo,Dec 20
49,SYNC Summit,2025-10-30,O West Orascom Development,None,"29.96718797603118, 30.9780625",giza,"30th, 31st of October & 1st of November"


In [93]:
cities = ["cairo",'giza']
year = 2025
month_num_start = 10
start_day = 1
start_date = datetime(year, month_num_start, start_day).strftime("%Y-%m-%d")
month_num_end = 12
end_day = 1
end_date = datetime(year, month_num_end, end_day).strftime("%Y-%m-%d")
simplified_data[
    (simplified_data["date"] >= start_date)
    & (simplified_data["date"] <= end_date)
    & (simplified_data["city"].isin(cities))
]

,event_name,date,venue_name,Time,venue_coordinates,city,original_date_text
9,SheArts,2025-10-02,Concert Hall (New Capital) - AUC Tahrir Square...,None,"30.042333968763398, 31.238579755550326",cairo,From 2nd to 5th Oct
11,Mina Nader X Theatro Arkan,2025-10-03,Theatro Arkan,09:00 PM,"30.020402636742446, 31.003588270211274",giza,Oct 03
12,Rachele Andriolo (Italy) / Sanyu (Norway),2025-10-05,AUC Tahrir Square,07:00 PM,"30.04304943486499, 31.236530904482507",cairo,Oct 05
13,El Masyada,2025-10-06,El Rihany Theater,8:00 PM,"30.05695814787614, 31.243279890052847",cairo,6 to 9 Oct
14,Tablet El Sitt,2025-10-10,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza,Oct 10
15,Saleh El-Nawawy X Boom Room,2025-10-13,"Boom Room, Open Air Mall, Madinaty",09:00 PM,"30.108738710312494, 31.6265184785103",cairo,Oct 13
26,Lena Chamamyan x Theatro Arkan,2025-10-02,Theatro Arkan,08:00 PM,"30.020402636742446, 31.003588270211274",giza,Oct 02
27,Yalla Masra7 3 Oct,2025-10-03,Must Theatre,08:00 PM,"29.99686480787293, 30.96529280980705",giza,Oct 03
28,Manal Mohei Eldin & Şirin Pancaroğlu / Regina ...,2025-10-04,AUC Tahrir Square,06:00 PM,"30.04304943486499, 31.236530904482507",cairo,Oct 04
29,3 Interrupted,2025-10-05,Rawabet Art Space,08:00 PM,"30.049483336333587, 31.238033769313294",cairo,Oct 05
